# Урок 3. Построение надежных схем валидации решения, оптимизация целевых метрик

##### Импорт библиотек

In [35]:
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
# import catboost as cb
import lightgbm as lgb

import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from sklearn.metrics import r2_score, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
%matplotlib inline

In [3]:
import warnings
warnings.simplefilter("ignore")
# warnings.filterwarnings('ignore')

## Основное задание:
Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы/минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - `IsFraud`. Рекомендуется использовать модели градиетного бустинга, реализация любая, гипепараметры любые.

**NB!** выборка `assignment_2_test.csv` - наш аналог лидерборда. Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. Для других целей использовать выборку запрещено!

Терминалогия, используемая в задании:
* обучающая выборка - выборка, которая передается в метод fit /train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.

In [4]:
train = pd.read_csv('./data/assignment2_data/assignment_2_train.csv')
leaderboard = pd.read_csv('./data/assignment2_data/assignment_2_test.csv')
target = 'isFraud'

print("train.shape = {} rows, {} cols".format(*train.shape))
print("leaderboard.shape = {} rows, {} cols".format(*leaderboard.shape))

train.shape = 180000 rows, 394 cols
leaderboard.shape = 100001 rows, 394 cols


In [5]:
train.head(n=3)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Задание 1. 
Сделать Hold-Out валидацию с разбиением, размер которого будет адекватным, по вашему мнению. Разбиение проводить по id-транзакции (`TransactionID`), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [15]:
numerical_features = train \
    .select_dtypes(include=[np.number]) \
    .drop(target, axis=1) \
    .fillna(0)

In [40]:
x_train, x_test, y_train, y_test = train_test_split(numerical_features,
                                                    train[target],
                                                    train_size=0.75,
                                                    shuffle=True,
                                                    random_state=42)

In [44]:
params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "n_estimators": 1000,
    "reg_lambda": 100,
    "max_depth": 4,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

model_xgb_numeric = xgb.XGBClassifier(**params)
model_xgb_numeric.fit(X=x_train,
                      y=y_train,
                      eval_set=[(x_train, y_train), (x_test, y_test)],
                      early_stopping_rounds=30,
                      eval_metric="auc",
                      verbose=20)

[0]	validation_0-auc:0.65337	validation_1-auc:0.63889
[20]	validation_0-auc:0.82380	validation_1-auc:0.81366
[40]	validation_0-auc:0.87181	validation_1-auc:0.86147
[60]	validation_0-auc:0.88704	validation_1-auc:0.87579
[80]	validation_0-auc:0.89311	validation_1-auc:0.88160
[100]	validation_0-auc:0.89777	validation_1-auc:0.88565
[120]	validation_0-auc:0.90061	validation_1-auc:0.88862
[140]	validation_0-auc:0.90337	validation_1-auc:0.89060
[160]	validation_0-auc:0.90402	validation_1-auc:0.89103
[175]	validation_0-auc:0.90402	validation_1-auc:0.89103


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=10, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=6, nthread=6,
              num_parallel_tree=1, random_state=27, reg_alpha=0, reg_lambda=100,
              scale_pos_weight=1, seed=27, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [66]:
train_score = roc_auc_score(y_train, model_xgb_numeric.predict(x_train))
test_score = roc_auc_score(y_test, model_xgb_numeric.predict(x_test))
LB_score = roc_auc_score(
    leaderboard[target], model_xgb_numeric.predict(
        leaderboard.select_dtypes(include=[np.number]).drop(target, axis=1).fillna(0)))

pd.DataFrame(data=[{'Train-score': train_score, 'Test-score': test_score, 
                    'LB-score': LB_score}], index=['xgb_numeric'])

,Train-score,Test-score,LB-score
xgb_numeric,0.66906,0.649814,0.632561


### Задание 2. Сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (`TransactionID`), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

### Задание 3. Построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

### Задание 4. Выполнить **Adversarial Validation**, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

### Задание 5. Сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных. Оценить качество на ЛБ, сделать выводы.

### Задание 6 (опциональное). Сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на ЛБ выборке с полученным доверительным интервалом. Сделать выводы.

### Задание 7 (совсем опциональное): в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).